# Quantum State Preparation Method Call Demo

This note book will give a simple walk-through of using the pre-implemented quantum state preparation function `qsp_qubit_eff` to execute the following QSP operation:
$$U_{\mathrm{QSP}} |0^n\rangle = |\psi\rangle = \frac{1}{\lVert \mathbf{x}\rVert}\sum_{i=0}^{2^{n-1}}x_i |i\rangle, x_i \in \mathbb{C}$$

In other words, we can prepare an arbitrary n-qubit quantum state (with complex coefficients on all basis) from a n-qubit zero state.

See https://github.com/guikaiwen/qubit_efficient_QSP/blob/main/%5BDetailed%20Implementation%5D%20QSP.ipynb for a detail implementation walk-through.

In [1]:
# Import Braket libraries
import braket
from braket.circuits import Circuit
from braket.aws import AwsDevice
from braket.devices import LocalSimulator

import numpy as np
import random

### Construct the QSP Circuit

We __import__ the quantum state preparation function `qsp_qubit_eff` from `qsp_circ_construction_qubit_efficient.py`, and some helper functions from `helper_functions.py`

In [2]:
from qsp_circ_construction_qubit_efficient import qsp_qubit_eff
from helper_functions import *

#### Generate Real Wave Function Vector

In order to define the QSP circuit, we would first need to specify the target wave function $\psi$ we want to get, expressed in the 1D-array form $[x_0, x_1, ..., x_{2^n - 1}]$. This wave function array will then be passed to the `qsp_qubit_eff` as the input parameter to define some quantum rotation gate angles.

In many applications it is sufficient to let the amplitude values be positive real numbers.

Let's generate a random normalized wave function vector array of size $2^3 = 8$ using the helper functions.

We will display the vector values in a column with all values rounded to 3 digits, using the helper functions.

In [3]:
%%time
np.random.seed(10) # fix a seed for data reproduction purpose, you can change this value to play with other results
n = 3 # number of qubits. We can encode 2^n number of values with only n qubits! You can change this parameter to see what will happen for larger QSP tasks
normalized_real_array = generate_normalized_real_array(n)
print(row_to_column_vector(round_to_three_significant_digits(normalized_real_array, 3))) # print out the wave function vector array for validation

[[0.317]
 [0.464]
 [0.331]
 [0.039]
 [0.014]
 [0.438]
 [0.156]
 [0.597]]
CPU times: user 221 µs, sys: 12 µs, total: 233 µs
Wall time: 235 µs


#### Call default construction function

With the wave function array stored in `normalized_real_array`, we can now call the QSP circuit construction methods to act on an empty Braket circuit, using the generated wave function array to define the rotation angles. You can see that the circuits are consisted by many multi-control Ry gates.

In [4]:
initialized_circ = qsp_qubit_eff(normalized_real_array)
print(initialized_circ)

T  : |   0    |   1    |2|   3    |4|   5    |6|   7    |8|   9    |10|11|   12   |13|
                                                                                      
q0 : -Ry(1.72)-C--------X-C--------X-C----------C--------X-C--------X--X--C--------X--
               |          |          |          |          |              |           
q1 : ----------Ry(1.91)---Ry(1.07)---C--------X-C--------X-C--------X-----C--------X--
                                     |          |          |              |           
q2 : --------------------------------Ry(2.63)---Ry(3.08)---Ry(0.23)-------Ry(1.94)----

T  : |   0    |   1    |2|   3    |4|   5    |6|   7    |8|   9    |10|11|   12   |13|


(Note that the `qsp_qubit_eff` function currently assert the input parameter `normalized_real_array` to be normalized and has length of power of 2)

### Quantum Circuit Execution

Now let's test our constructed circuit!

We execute the constructed circuit using the Braket's local simulator.

In [5]:
%%time
braket_device = LocalSimulator() # define the simulator
initialized_circ.state_vector() # convert the circuit to state vector
braket_state_vector_result = braket_device.run(initialized_circ, shots=0).result().values[0] # extract the result
print(row_to_column_vector(round_to_three_significant_digits(braket_state_vector_result, 3))) # print out the resulted state vector

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.


[[0.317+0.j]
 [0.464+0.j]
 [0.331+0.j]
 [0.039+0.j]
 [0.014+0.j]
 [0.438+0.j]
 [0.156+0.j]
 [0.597+0.j]]
CPU times: user 36.1 ms, sys: 1.94 ms, total: 38 ms
Wall time: 37.3 ms


Let's see the actual difference between the expected amplitude vector vs what is actually produced

In [6]:
print(row_to_column_vector(np.array(braket_state_vector_result) - np.array(normalized_real_array)))

[[-5.55111512e-17+0.j]
 [-2.22044605e-16+0.j]
 [-5.55111512e-17+0.j]
 [-6.93889390e-18+0.j]
 [-6.47051857e-16+0.j]
 [ 1.11022302e-16+0.j]
 [ 2.49800181e-16+0.j]
 [ 1.11022302e-16+0.j]]


The result matches perfectly with the desired output!

#### [Advanced] Generate Complex Arbitrary Quantum State

Now let's generate a size $2^3 = 8$ random array with complex values:

In [7]:
%%time
np.random.seed(10) # fix a seed for data reproduction purpose, you can change this value to play with other results
n = 3 # number of qubits. We can encode 2^n number of values with n qubits!
normalized_complex_array = generate_normalized_complex_array(n)
print(row_to_column_vector(round_to_three_significant_digits(normalized_complex_array, 3)))

[[ 0.222-0.271j]
 [-0.392-0.337j]
 [ 0.109+0.152j]
 [ 0.204+0.371j]
 [-0.001-0.406j]
 [-0.225+0.01j ]
 [-0.247+0.256j]
 [ 0.213+0.092j]]
CPU times: user 456 µs, sys: 55 µs, total: 511 µs
Wall time: 494 µs


We can construct the circuit using the same `qsp_qubit_eff` function. Notice that there is another layer of multi-control Rz gate added on top of the multi-control Ry gate to pump in the phase values.

In [8]:
initialized_circ = qsp_qubit_eff(normalized_complex_array)
print(initialized_circ)

T  : |   0    |   1    |   2    |   3    |4|   5    |   6    |7|   8    |    9    |10|   11   |   12   |13|   14   |   15   |16|17|   18   |   19    |20|
                                                                                                                                                         
q0 : -Ry(1.36)-Rz(1.39)-C--------C--------X-C--------C--------X-C--------C------------C--------C--------X--C--------C--------X--X--C--------C---------X--
                        |        |          |        |          |        |            |        |           |        |              |        |            
q1 : -------------------Ry(1.48)-Rz(0.61)---Ry(1.28)-Rz(2.67)---C--------C---------X--C--------C--------X--C--------C--------X-----C--------C---------X--
                                                                |        |            |        |           |        |              |        |            
q2 : -----------------------------------------------------------Ry(1.16)-Rz(

We execute the constructed circuit using the Braket's local simulator:

In [9]:
%%time
braket_device = LocalSimulator() # define the simulator
initialized_circ.state_vector() # convert the circuit to state vector
braket_state_vector_result = braket_device.run(initialized_circ, shots=0).result().values[0] # extract the result
print(row_to_column_vector(round_to_three_significant_digits(braket_state_vector_result, 3))) # print out the resulted state vector

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.


[[ 0.109-0.333j]
 [-0.488-0.172j]
 [ 0.157+0.102j]
 [ 0.324+0.272j]
 [-0.148-0.378j]
 [-0.206+0.091j]
 [-0.137+0.328j]
 [ 0.232+0.008j]]
CPU times: user 15 ms, sys: 984 µs, total: 16 ms
Wall time: 15.2 ms


Notice that the circuit state vector output looks very different from the generated vector input. Why is this the case? This is because we have a global phase difference! In other words, instead of having the exact state $|\psi\rangle$, we instead have $e^{i\theta}|\psi\rangle$. This additional global phase does not carry any physical meanings, nor does it affect the final measurement results on the real quantum computer.

We can perform the test below to see if circuit state vector result is the same as the input vector but only with a global phase difference. For the circuit state vector result $|\psi'\rangle = e^{i\theta}|\psi\rangle$ and input vector $|\psi\rangle$, we should expect $||\langle\psi'|\psi\rangle|| = 1$

In [10]:
print(np.abs(np.dot(normalized_complex_array, np.conj(braket_state_vector_result))))

1.0000000000000002


We can also try to compute the phase difference $\theta_i$ between each pair of basis:

In [11]:
phase_v1 = np.angle(normalized_complex_array)
phase_v2 = np.angle(braket_state_vector_result)

# Compute the global phase difference between v1 and v2
global_phase_difference = phase_v1 - phase_v2
global_phase_difference = (global_phase_difference + np.pi) % (2 * np.pi) - np.pi
print(global_phase_difference)

[0.37114178 0.37114178 0.37114178 0.37114178 0.37114178 0.37114178
 0.37114178 0.37114178]


You can see that all the phase differences are the same across all values (and thus a global phase)!

Now we can add the phase correction to the circuit state vector output:

In [12]:
global_phase_corrected_braket_state_vector_result = braket_state_vector_result * np.exp(1j * global_phase_difference[0])
print(row_to_column_vector(round_to_three_significant_digits(global_phase_corrected_braket_state_vector_result, 3)))

[[ 0.222-0.271j]
 [-0.392-0.337j]
 [ 0.109+0.152j]
 [ 0.204+0.371j]
 [-0.001-0.406j]
 [-0.225+0.01j ]
 [-0.247+0.256j]
 [ 0.213+0.092j]]


This is now the same as the input vector.

Printing out the actual differences:

In [13]:
print(row_to_column_vector(global_phase_corrected_braket_state_vector_result - normalized_complex_array))

[[-2.77555756e-17-5.55111512e-17j]
 [ 1.11022302e-16+0.00000000e+00j]
 [ 6.93889390e-17+1.11022302e-16j]
 [ 0.00000000e+00+0.00000000e+00j]
 [-3.75133952e-17-1.66533454e-16j]
 [-8.32667268e-17+2.42861287e-17j]
 [-5.55111512e-17-5.55111512e-17j]
 [ 0.00000000e+00+1.38777878e-17j]]


#### [Optional] Generate Sparse Arbitrary Quantum State

Now, let's create a a w-like state: $|\psi\rangle = \alpha|001\rangle + \beta|010\rangle + \gamma|100\rangle$ and see what happens.

We first create a 1d array of size $2^3 = 8$ with arbitrary $\alpha$, $\beta$, and $\gamma$ values at their proper postion, and fill the rest postions with $0$s:

In [14]:
random.seed(10)
w_like_state = generate_normalized_real_sparse_array(3, [7])
print(row_to_column_vector(round_to_three_significant_digits(w_like_state, 3)))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]


Now, call the same `qsp_qubit_eff` function to generate the circuit:

In [15]:
initialized_circ = qsp_qubit_eff(w_like_state)
print(initialized_circ)

T  : |   0    |   1    |   2    |
                                 
q0 : -Ry(3.14)-C--------C--------
               |        |        
q1 : ----------Ry(3.14)-C--------
                        |        
q2 : -------------------Ry(3.14)-

T  : |   0    |   1    |   2    |


We can see that we only need $3$ multi-control Ry gates instead of $7$.

Now, execute the circuit on the Braket simulator:

In [16]:
initialized_circ.state_vector() # convert the circuit to state vector
braket_state_vector_result = braket_device.run(initialized_circ, shots=0).result().values[0] # extract the result
print(row_to_column_vector(round_to_three_significant_digits(braket_state_vector_result, 3))) # print out the resulted state vector

This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.


[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [1.+0.j]]


The result matches the desired output.

<span style="color:orange;"> As mentioned in the beginning, there could be some further optimization to further reduce the classical pre-comuting cost, and possibly some quantum gate cost for general arbitrary sparse states. </span>

In [17]:
import braket._sdk as braket_sdk
braket_sdk.__version__

'1.55.0'

In [18]:
import sys
print(sys.version)

3.10.11 (main, May 17 2023, 14:30:36) [Clang 14.0.6 ]
